# Structured Streaming using Python DataFrames API

Apache Spark 2.0 adds the first version of a new higher-level stream processing API, Structured Streaming. In this notebook we are going to take a quick look at how to use DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a **Spark 2.x** cluster.

## Sample Data

In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path,name,size
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008


Hay alrededor de 50 archivos JSON en el directorio. Veamos qué contiene cada archivo JSON.

In [0]:
%fs head /databricks-datasets/structured-streaming/events/file-0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

Cada línea del archivo contiene un registro JSON con dos campos: tiempo y acción. Vamos a analizar estos archivos de forma interactiva.

## Batch/Interactive Processing
El primer paso es consultar los datos de forma interactiva. Para hacerlo, se define un DataFrame estático.

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

#Como ya conocemos el formato de los datos, se define el esquema para acelerar el procesamiento
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Se define el DataFrame estático que toma los datos y el esquema definido previamente. Sirve para representar el contenido de los archivos JSON
staticInputDF = (spark.read.schema(jsonSchema).json(inputPath))
display(staticInputDF)

time,action
2016-07-28T04:19:28.000+0000,Close
2016-07-28T04:19:28.000+0000,Close
2016-07-28T04:19:29.000+0000,Open
2016-07-28T04:19:31.000+0000,Close
2016-07-28T04:19:31.000+0000,Open
2016-07-28T04:19:31.000+0000,Open
2016-07-28T04:19:32.000+0000,Close
2016-07-28T04:19:33.000+0000,Close
2016-07-28T04:19:35.000+0000,Close
2016-07-28T04:19:36.000+0000,Open


Ahora vamos a calcular el número de acciones de "abrir" y "cerrar" en una ventana de tiempo 1 hora. Para hacer esto, se agrupará por la columna de acción y se indicará explicitamente la ventana de 1 hora sobre la columna de tiempo.

In [0]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (staticInputDF.groupBy(staticInputDF.action,window(staticInputDF.time, "1 hour")).count())
staticCountsDF.cache()

# El DataFrame se crea como una tabla
staticCountsDF.createOrReplaceTempView("static_counts")

Ahora ya podemos usar SQL directamente para consultar la tabla. Por ejemplo, aquí están los recuentos totales de todas las horas.

In [0]:
%sql select action, sum(count) as total_count from static_counts group by action

action,total_count
Close,50000
Open,50000


A continuación generamos una línea de tiempo de recuentos de eventos

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


Observa los dos extremos de la gráfica. 
Las acciones de cierre se generan de tal manera que son posteriores a las acciones de apertura correspondientes, por lo que hay más "aperturas" al principio y más "cierres" al final.

## Stream Processing: emulando una transmisión continúa de datos

Dado que ya analizados los datos, ahora vamos a convertir en una consulta de transmisión que se actualiza a medida que llegan los datos.

¿Cómo?
Dado que solo tenemos un conjunto estático de archivos, se va a emular un flujo de ellos leyendo un archivo a la vez, en el orden cronológico en que fueron creados.

In [0]:
from pyspark.sql.functions import *

#Recordemos la función creada anteriormente
#staticInputDF = (spark.read.schema(jsonSchema).json(inputPath))

# Ahora en lugar de usar `read` se usará `readStream`.
streamingInputDF = (spark.readStream.schema(jsonSchema)
                    .option("maxFilesPerTrigger", 1)  # Trata una secuencia de archivos como un flujo de datos
                                                      #tomando un archivo a la vez
                    .json(inputPath))

#Recordemos la consulta anterior
#staticCountsDF = (staticInputDF.groupBy(staticInputDF.action,window(staticInputDF.time, "1 hour")).count())

#Generamos la misma consulta
streamingCountsDF = (streamingInputDF.groupBy(streamingInputDF.action, window(streamingInputDF.time, "1 hour"))
                      .count())

# Ahora definimos que `streamingCountsDF`es un DataFrame para flujo de datos, y retorna `True`
streamingCountsDF.isStreaming  #streaming Dataframe 

Out[46]: True

¿Qué sigue?
Vamos a consultar los datos de ¡manera interactiva!

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (streamingCountsDF.writeStream
                          .format("memory")#almacena la tabla en memoria
                          .queryName("counts")#nombre de la tabla en memoria
                          .outputMode("complete")#todas las cuentas deben estar en la tabla
                          .start())

`query` es un identificador de la consulta de transmisión que se ejecuta en segundo plano. 
Esta consulta extrae los archivos continuamente y actualiza los recuentos en ventana.

La barra de progreso muestra que la consulta está activa. Además, si expande los `conteos` se observará la cantidad de archivos que ya han procesado.

In [0]:
from time import sleep
sleep(5)  # esperemos un poco para iniciar el conteo

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,176
Open,Jul-26 05:00,289


Observemos la línea de tiempo de recuentos en ventana (similar a la estática anterior) acumulándose. 

Si seguimos ejecutando esta consulta interactiva repetidamente, se observará los últimos recuentos actualizados que la consulta de transmisión está actualizando en segundo plano.

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


In [0]:
sleep(5)  # wait a bit more for more data to be computed

Analicemos el número de aperturas y cierre

In [0]:
%sql select action, sum(count) as total_count from counts group by action order by action

action,total_count
Close,6488
Open,7512


In [0]:
#query.stop()